In [33]:
import sys
sys.path.append('../code/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from nilearn import datasets, plotting

from mamba_ssm import Mamba

import utils
from train_ann_sweep import get_generators
import pickle
from train_ann_sweep import get_generators

device = torch.device("cuda:0")
# device = torch.device("cpu")

In [34]:
atlas = datasets.fetch_atlas_msdl()
# Loading atlas image stored in 'maps'
atlas_filename = atlas["maps"]
# Loading atlas data stored in 'labels'
labels = atlas["labels"]

df = pd.read_pickle('../data/developmental_df.pkl')
n_subjects = len(np.unique(df['subj'].values))

time_series = df.iloc[:,:-1].values

# pca = PCA(n_components=39)
# points = pca.fit_transform(time_series)


In [35]:
with open('../data/ann_bottleneck_sweep_results.pkl', 'rb') as f:
    metadata = pickle.load(f)
print(metadata['bottleneck_values'])

cv_dict = metadata['bottleneck_1']['fold_0']['metadata']['cv_dict']

[1, 3, 5, 7, 9, 11, 13, 15]


In [120]:
criterion = nn.MSELoss()

def get_autoencoder_mse(training_set, validation_set, fold=0, bottleneck=15):
    fold
    input_size = 39
    encoder_layer_size = [500, 300, 100]
    decoder_layer_size = encoder_layer_size[::-1]


    model = utils.model_ann_autoencoder(input_size, encoder_layer_size, decoder_layer_size, bottleneck=bottleneck).to(device)
    state_dict = torch.load(f'../models/ann/bottleneck/ann_fold{fold}_bottleneck{bottleneck}.pt')
    model.load_state_dict(state_dict)

    model.eval()
    with torch.no_grad():
        y_pred_train = model(training_set[:].float().to(device))[0].detach().cpu()
        y_pred_val = model(validation_set[:].float().to(device))[0].detach().cpu()

    return criterion(training_set[:], y_pred_train).numpy(), criterion(validation_set[:], y_pred_val).numpy(),

   

def get_pca_mse(training_set, validation_set, fold=0, bottleneck=15):

    pca = PCA(n_components=bottleneck)
    batch_num, num_times, num_features = training_set[:].shape

    train_points = pca.fit_transform(training_set[:].numpy().reshape(-1, num_features))
    val_points = pca.transform(validation_set[:].numpy().reshape(-1, num_features))

    y_pred_train = torch.tensor(pca.inverse_transform(train_points).reshape(training_set[:].shape))
    y_pred_val = torch.tensor(pca.inverse_transform(val_points).reshape(validation_set[:].shape))

    return criterion(training_set[:], y_pred_train).numpy(), criterion(validation_set[:], y_pred_val).numpy()


    # explained_var = np.cumsum(pca.explained_variance_ratio_)
    # explained_var = np.concatenate([[0], explained_var])
    # thresh_pos = np.argmin(np.abs(explained_var - 0.8))




In [121]:
fold = 0
bottleneck = 15

generators, data_arrays = get_generators(df, cv_dict, fold=fold)
training_generator, _, validation_generator, _ = generators
training_set, _, validation_set = data_arrays

In [129]:
get_autoencoder_mse(training_set, validation_set, fold=1, bottleneck=9)

(array(0.44662055), array(0.45390814))

In [130]:
get_pca_mse(training_set, validation_set, fold=1, bottleneck=9)

(array(0.34506669), array(0.33747592))

In [82]:
len(df)

26040